In this project I will try to predict emotions from audio 
Dataset: https://drive.google.com/file/d/11e1JKXddm5cMyIxRVciYZGFTyXJmNq1l/view?usp=sharing

In [1]:
import librosa 
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
def extract_feature(file_name,mfcc,chroma,mel):
    with soundfile.SoundFile(file_name) as sound_file:
        x=sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(x))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=x,sr=sample_rate,n_mfcc=40).T,axis=0)
            result=np.hstack((result,mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft,sr=sample_rate).T,axis=0)
            result=np.hstack((result,mfccs))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(x,sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    
    return result
    




In [3]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

observed_emotions=['calm', 'happy', 'fearful', 'disgust']


In [8]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file,mfcc=True,chroma=True,mel=True)
        x.append(feature)
        y.append(emotion)
    
    return train_test_split(np.array(x),y,test_size=test_size,random_state=42)

In [9]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

C:\Users\Dell\AppData\Local\Temp/ipykernel_20400/1990291355.py:15: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x,sr=sample_rate).T,axis=0)
C:\Users\Dell\AppData\Local\Temp/ipykernel_20400/1990291355.py:15: FutureWarning: Pass y=[0.0000000e+00 3.0517578e-05 3.0517578e-05 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x,sr=sample_rate).T,axis=0)
C:\Users\Dell\AppData\Local\Temp/ipykernel_20400/1990291355.py:15: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00 -3.0517578e-05 ... -3.0517578e-05
 -3.0517578e-05 -3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x,sr=sample_rate).T,axis=0)
C:\Users\De

In [10]:
x_train.shape

(576, 208)

In [12]:
model =MLPClassifier(alpha=0.01,batch_size=256,epsilon=1e-08,hidden_layer_sizes=(300,),learning_rate="adaptive",max_iter=500)

In [13]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [14]:
y_pred=model.predict(x_test)

In [15]:
accuracy=accuracy_score(y_true=y_test,y_pred=y_pred)
accuracy

0.5989583333333334